In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import math
import re

In [2]:
import json
from scrapfly import ScrapeApiResponse, ScrapeConfig, ScrapflyClient

In [3]:
client = ScrapflyClient(key="scp-live-250eae03bd25400bab7372436d0431da")

In [148]:
def find_companies(query: str):
    """find company Glassdoor ID and name by query. e.g. "ebay" will return "eBay" with ID 7853"""
    try: 
        result = client.scrape(
            ScrapeConfig(
                url=f"https://www.glassdoor.com/searchsuggest/typeahead?numSuggestions=8&source=GD_V2&version=NEW&rf=full&fallback=token&input={query}",
                country="US",
                asp=True,
                cookies={"tldp":"1"},  # sets location to US
            )
        )
    
        data = json.loads(result.content)
        return data[0]["suggestion"], data[0]["employerId"]
    except (ValueError, IndexError):
        return -1
        
        
    


In [61]:
master_list = pd.read_excel("firms_list.xlsx", usecols=['EXEC_FULLNAME', 'CONAME'])

In [10]:
firm_list.head()

,EXEC_FULLNAME,CONAME
0,"John McClain Holmes, III",AAR CORP
1,"William Douglas Parker, BEC, M.B.A.",AMERICAN AIRLINES GROUP INC
2,"Robert D. Isom, Jr.",AMERICAN AIRLINES GROUP INC
3,"Donald E. Brandt, CPA",PINNACLE WEST CAPITAL CORP
4,Jeffrey B. Guldner,PINNACLE WEST CAPITAL CORP


In [13]:
firm_list['glassdor_id'] = firm_list['CONAME'].apply(find_companies)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [62]:
firm_list

,EXEC_FULLNAME,CONAME
0,"John McClain Holmes, III",AAR CORP
1,"William Douglas Parker, BEC, M.B.A.",AMERICAN AIRLINES GROUP INC
2,"Robert D. Isom, Jr.",AMERICAN AIRLINES GROUP INC
3,"Donald E. Brandt, CPA",PINNACLE WEST CAPITAL CORP
4,Jeffrey B. Guldner,PINNACLE WEST CAPITAL CORP
...,...,...
2137,Matthew Craig Kapusta,UNIQURE NV
2138,"Paul Anthony Novelly, II",FUTUREFUEL CORP
2139,Bhavesh V. Patel,LYONDELLBASELL INDUSTRIES NV
2140,Peter Z. E. Vanacker,LYONDELLBASELL INDUSTRIES NV


In [16]:
# from bs4 import BeautifulSoup
# from urllib.request import Request, urlopen

In [17]:
#create a function to scrape any Glassdoor company review page
#the code still works when I run it on 7 Sep, 2021, but the html content of Glassdoor webpages changes all the time
#please inspect the webpage and make the necessary changes to the html tags if any of the list returns empty

# def review_scraper(url):
#   #scraping the web page content
#   hdr = {'User-Agent': 'Mozilla/5.0'}
#   req = Request(url,headers=hdr)
#   page = urlopen(req)
#   soup = BeautifulSoup(page, "html.parser") 

#   #define some lists
#   Summary=[]
#   Date_n_JobTitle=[]
#   Date=[]
#   JobTitle=[]
#   AuthorLocation=[]
#   OverallRating=[]
#   Pros=[]
#   Cons=[]  

#   #get the Summary
#   for x in soup.find_all('h2', {'class':'mb-xxsm mt-0 css-5j5djr'}):
#     Summary.append(x.text)

#   #get the Posted Date and Job Title
#   for x in soup.find_all('span', {'class':'authorJobTitle middle common__EiReviewDetailsStyle__newGrey'}):
#     Date_n_JobTitle.append(x.text)

#   #get the Posted Date
#   for x in Date_n_JobTitle:
#     Date.append(x.split(' -')[0])

#   #get Job Title
#   for x in Date_n_JobTitle:
#     JobTitle.append(x.split(' -')[1])

#   #get Author Location
#   for x in soup.find_all('span', {'class':'authorLocation'}):
#     AuthorLocation.append(x.text)

#   #get Overall Rating
#   for x in soup.find_all('span', {'class':'ratingNumber mr-xsm'}):
#     OverallRating.append(float(x.text))

#   #get Pros
#   for x in soup.find_all('span', {'data-test':'pros'}):
#     Pros.append(x.text)

#   #get Cons
#   for x in soup.find_all('span', {'data-test':'cons'}):
#     Cons.append(x.text)

#   #putting everything together
#   Reviews = pd.DataFrame(list(zip(Summary, Date, JobTitle, AuthorLocation, OverallRating, Pros, Cons)), 
#                     columns = ['Summary', 'Date', 'JobTitle', 'AuthorLocation', 'OverallRating', 'Pros', 'Cons'])
  
#   return Reviews

In [311]:
mini_dataframe = firm_list.iloc[1400:]

In [312]:
mini_dataframe

,CONAME
2037,FABRINET
2038,PHILIP MORRIS INTERNATIONAL
2040,HILLENBRAND INC
2042,LOGMEIN INC
2043,QUANEX BUILDING PRODUCTS
...,...
2136,KITE REALTY GROUP TRUST
2137,UNIQURE NV
2138,FUTUREFUEL CORP
2139,LYONDELLBASELL INDUSTRIES NV


In [313]:
mini_dataframe['glassdor_id'] = mini_dataframe['CONAME'].apply(find_companies)

/var/folders/_d/zb748hgj43q6ghhpp4d7hm24nrhl5j/T/ipykernel_67799/2379397718.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mini_dataframe['glassdor_id'] = mini_dataframe['CONAME'].apply(find_companies)


In [152]:
#firm_list = firm_list.drop(['EXEC_FULLNAME'], axis='columns')

In [64]:
firm_list= firm_list.drop_duplicates()

In [69]:
firm_list =firm_list.iloc[16:]

In [75]:
firm_list.reset_index()

,CONAME
21,US ECOLOGY INC
22,AMERICAN ELECTRIC POWER CO
23,AMERICAN EXPRESS CO
24,AFLAC INC
25,AMERICAN INTERNATIONAL GROUP
...,...
2136,KITE REALTY GROUP TRUST
2137,UNIQURE NV
2138,FUTUREFUEL CORP
2139,LYONDELLBASELL INDUSTRIES NV


In [314]:
master_list = master_list.merge(mini_dataframe, how='left', on=['CONAME'], validate='m:1')

In [316]:
master_list['glassdor_id'] = master_list['glassdor_id_x'].combine_first(master_list['glassdor_id_y'])

In [317]:
master_list = master_list.drop(['glassdor_id_x', 'glassdor_id_y'], axis='columns')

In [328]:
master_list.describe()

,EXEC_FULLNAME,CONAME,glassdor_id,company_name,glassdoor_id
count,2142,2142,2142,2116,2116
unique,2123,1496,1252,1251,1148
top,Patricia Kessler Poppe,SANMINA CORP,"(Corporate Recruiter, 0)",Corporate Recruiter,0
freq,2,4,134,134,381


In [323]:
master_list.dtypes

EXEC_FULLNAME    object
CONAME           object
glassdor_id      object
dtype: object

In [326]:
master_list[['company_name', 'glassdoor_id']] = master_list['glassdor_id'].apply(lambda x: pd.Series(x) if isinstance(x, tuple) and len(x) == 2 else pd.Series([None, None]))

In [329]:
master_list= master_list.drop(['glassdor_id'], axis='columns')

,EXEC_FULLNAME,CONAME,company_name,glassdoor_id


In [341]:
unretreived_data = master_list.loc[master_list['company_name'].isna()]

In [344]:
unretreived_data.append(master_list.loc[master_list['company_name'] == "Corporate Recruiter"])

AttributeError: 'DataFrame' object has no attribute 'append'